# HTML → LLM: Preprocessing for Accessibility Auditing

**Converting a 1.9 MB (500 k+ token) web page into structured, LLM-ready payloads**

---

This notebook documents the full pipeline for transforming a real-world HTML file —
the [Vision Aid](https://visionaid.org) homepage — into compact JSON payloads that a
language model can reason over for WCAG accessibility evaluation.

Three checklists are covered, each with its own extractor and prompt library:

| Checklist | File | Covers |
|---|---|---|
| 01 — Semantic Structure | `semantic_checklist_01.py` | Headings, links, landmarks, tables, iframes |
| 02 — Forms | `forms_checklist_02.py` | Labels, groups, required fields, instructions |
| 03 — Non-text Content | `nontext_checklist_03.py` | Images, SVG, icon fonts, media |

In [29]:
from pathlib import Path
import json, re, sys
from bs4 import BeautifulSoup
from collections import Counter
import importlib.util

# ── Paths ─────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR  = Path(".").resolve()
PROJECT_ROOT  = (NOTEBOOK_DIR / "../..").resolve()
HTML_FILE     = PROJECT_ROOT / "test_files" / "home.html"
PREPROCESSING = NOTEBOOK_DIR
PROMPTS_DIR   = PROJECT_ROOT / "processing_scripts" / "llm"

def load_extractor(name):
    # Dynamically load an extractor module by filename stem.
    path = PREPROCESSING / f"{name}.py"
    spec = importlib.util.spec_from_file_location(name, path)
    mod  = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

def estimate_tokens(text):
    return max(1, len(text) // 4)

print(f"HTML file : {HTML_FILE}  (exists={HTML_FILE.exists()})")
print(f"Extractors: {PREPROCESSING}")
print(f"Prompts   : {PROMPTS_DIR}")

HTML file : /Users/andrew/git/visionaid-a11y-llm-audit/test_files/home.html  (exists=True)
Extractors: /Users/andrew/git/visionaid-a11y-llm-audit/processing_scripts/llm_preprocessing
Prompts   : /Users/andrew/git/visionaid-a11y-llm-audit/processing_scripts/llm


---
## Step 1 — Measuring the Problem

How large is the raw HTML? Can it fit in an LLM context window as-is?

In [31]:
raw_text   = HTML_FILE.read_text(encoding="utf-8")
raw_bytes  = HTML_FILE.stat().st_size
raw_chars  = len(raw_text)
raw_lines  = raw_text.count("\n")
raw_tokens = raw_chars // 4

print("=" * 56)
print("  RAW HTML FILE METRICS")
print("=" * 56)
print(f"  File size    : {raw_bytes:>10,} bytes  ({raw_bytes/1024/1024:.2f} MB)")
print(f"  Characters   : {raw_chars:>10,}")
print(f"  Lines        : {raw_lines:>10,}")
print(f"  Est. tokens  : {raw_tokens:>10,}  (4 chars / token heuristic)")
print()

windows = [
    ("GPT-4o / GPT-4o-mini",       128_000),
    ("Claude Sonnet 4.6 (200 k)",  200_000),
]
print(f"  {'Model':<36} {'Limit':>8}   {'% used':>7}   Fits?")
print(f"  {'-'*58}")
for name, limit in windows:
    pct  = raw_tokens / limit * 100
    icon = "✓" if raw_tokens <= limit else "✗"
    print(f"  {name:<36} {limit:>8,}   {pct:>6.0f}%   {icon}")

  RAW HTML FILE METRICS
  File size    :  1,948,296 bytes  (1.86 MB)
  Characters   :  1,947,379
  Lines        :     14,626
  Est. tokens  :    486,844  (4 chars / token heuristic)

  Model                                   Limit    % used   Fits?
  ----------------------------------------------------------
  GPT-4o / GPT-4o-mini                  128,000      380%   ✗
  Claude Sonnet 4.6 (200 k)             200,000      243%   ✗


---
## Step 2 — What's Taking Up All Those Tokens?

In [34]:
soup = BeautifulSoup(raw_text, "lxml")

def char_size(tags):
    return sum(len(str(el)) for el in tags)

sections = {
    "<style> blocks  (inline CSS)":    char_size(soup.find_all("style")),
    "<script> blocks (JS / JSON-LD)":  char_size(soup.find_all("script")),
    "<svg> elements":                  char_size(soup.find_all("svg")),
    "<img> tags":                      char_size(soup.find_all("img")),
    "<a> link tags":                   char_size(soup.find_all("a")),
    "<div> / <section> wrappers":      char_size(soup.find_all(["div", "section"])),
}
accounted = sum(sections.values())
sections["Text nodes / other"] = max(0, raw_chars - accounted)

print(f"  {'Element type':<34} {'Chars':>10}  {'~Tokens':>8}  {'% of file':>9}")
print("  " + "-" * 68)
for label, chars in sorted(sections.items(), key=lambda x: -x[1]):
    toks = chars // 4
    pct  = chars / raw_chars * 100
    bar  = "█" * int(pct / 2)
    print(f"  {label:<34} {chars/10:>10,}  {toks/10:>8,}  {pct/10:>8.1f}%  {bar}")
print("  " + "-" * 68)
print(f"  {'TOTAL':<34} {raw_chars:>10,}  {raw_chars//4:>8,}  {'100.0%':>9}")

  Element type                            Chars   ~Tokens  % of file
  --------------------------------------------------------------------
  <div> / <section> wrappers         1,914,066.0  478,516.5      98.3%  ███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
  <a> link tags                        56,954.0  14,238.5       2.9%  ██████████████
  <img> tags                           17,510.9   4,377.7       0.9%  ████
  <style> blocks  (inline CSS)          9,639.9   2,409.9       0.5%  ██
  <script> blocks (JS / JSON-LD)        9,466.9   2,366.7      

---
## Checklist 01 — Semantic Structure & Navigation

**Extractor**: `semantic_checklist_01.py` | **Prompts**: `semantic_checklist_01.txt`

Targets the elements needed to evaluate page structure, navigation, and
content labelling against WCAG 1.3.1, 2.4.x, and 4.1.x.

| Section | WCAG | What the LLM judges |
|---|---|---|
| `language` | 3.1.1 | Is lang set? (programmatic — included for completeness) |
| `page_title` + `headings` | 2.4.2, 1.3.1 | Are title/headings meaningful and logical? |
| `flagged_links` | 2.4.4 | Are short/generic link texts clear out of context? |
| `landmarks` | 1.3.6 | Are multiple navs differentiated? Is structure appropriate? |
| `images` (basic) | 1.1.1 | Quick alt-text presence check (detailed eval in CL03) |
| `forms` (basic) | 1.3.1 | Label presence check (detailed eval in CL02) |
| `tables` / `iframes` | 1.3.1, 4.1.2 | Caption/header clarity, iframe title quality |

In [35]:
cl01     = load_extractor("semantic_checklist_01")
cl01_pay = cl01.extract(str(HTML_FILE))
cl01_json = json.dumps(cl01_pay, indent=2)
cl01_tok  = estimate_tokens(cl01_json)

print("Checklist 01 — Section summary")
print(f"  {'Section':<22}  {'Items':>6}")
print(f"  {'-'*32}")
for key, val in cl01_pay.items():
    if isinstance(val, list):
        print(f"  {key:<22}  {len(val):>6}")
    elif key == "images":
        total = sum(len(v) for v in val.values())
        miss  = len(val["missing_alt"])
        empty = len(val["empty_alt"])
        has   = len(val["has_alt"])
        print(f"  {key:<22}  {total:>6}  (missing={miss}, empty={empty}, has_alt={has})")
    elif isinstance(val, dict):
        print(f"  {key:<22}  {'—':>6}")
    else:
        print(f"  {key:<22}  {repr(val)[:20]:>20}")
print()
print(f"  Total payload : {len(cl01_json):>8,} chars  |  ~{cl01_tok:>6,} tokens")

Checklist 01 — Section summary
  Section                  Items
  --------------------------------
  language                             'en-US'
  page_title                   —
  headings                   280
  images                     306  (missing=0, empty=134, has_alt=172)
  flagged_links               76
  forms                       16
  buttons                     15
  landmarks                   62
  tables                       0
  iframes                     15

  Total payload :   70,644 chars  |  ~17,661 tokens


In [36]:
# Token budget per CL01 prompt
cl01_slices = {
    "1. Page Title":         {"page_title": cl01_pay["page_title"]},
    "2. Heading Structure":  {"page_title": cl01_pay["page_title"],
                              "headings":   cl01_pay["headings"]},
    "3. Link Clarity":       {"flagged_links": cl01_pay["flagged_links"]},
    "4. Table Semantics":    {"tables":    cl01_pay["tables"]},
    "5. Iframe Titles":      {"iframes":   cl01_pay["iframes"]},
    "6. Landmark Structure": {"landmarks": cl01_pay["landmarks"]},
    "7. Combined Summary":   cl01_pay,
}

print("Checklist 01 — Token budget per prompt")
print(f"  {'Prompt':<26}  {'~Tokens':>8}")
print(f"  {'-'*38}")
for name, data in cl01_slices.items():
    t = estimate_tokens(json.dumps(data, indent=2))
    print(f"  {name:<26}  {t:>8,}")

Checklist 01 — Token budget per prompt
  Prompt                       ~Tokens
  --------------------------------------
  1. Page Title                     30
  2. Heading Structure           7,139
  3. Link Clarity                1,283
  4. Table Semantics                 4
  5. Iframe Titles                 772
  6. Landmark Structure          1,028
  7. Combined Summary           17,661


---
## Checklist 02 — Form Labels, Instructions & Groups

**Extractor**: `forms_checklist_02.py` | **Prompts**: `forms_checklist_02.txt`

Provides the LLM with the full label context for every form control — not
just whether a label exists, but *what* the label says, *how* it's associated,
and *what instructions* are linked. This enables qualitative judgment that
programmatic tools cannot make.

| Section | WCAG | What the LLM judges |
|---|---|---|
| `fields.effective_label` | 1.3.1, 2.4.6 | Is the label descriptive and meaningful? |
| `fields.label_source` | 1.3.1 | Is `placeholder_only` being used as a label? |
| `fields.instructions` | 3.3.2 | Are aria-describedby instructions clear and helpful? |
| `fields.required` | 3.3.2 | Is required status communicated in the label/instructions? |
| `groups.legend` | 1.3.1 | Does the fieldset legend provide sufficient group context? |

In [37]:
cl02     = load_extractor("forms_checklist_02")
cl02_pay = cl02.extract(str(HTML_FILE))
cl02_json = json.dumps(cl02_pay, indent=2)
cl02_tok  = estimate_tokens(cl02_json)

forms = cl02_pay["forms"]
all_fields = [f for form in forms for f in form["fields"]]
all_groups = [g for form in forms for g in form["groups"]]

# Label source distribution
src_counts = Counter(f["label_source"] for f in all_fields)
required   = [f for f in all_fields if f["required"]]
with_instr = [f for f in all_fields if f["instructions"]]

print("Checklist 02 — Forms summary")
print(f"  Total forms            : {len(forms)}")
print(f"  Total fields           : {len(all_fields)}")
print(f"  Total groups           : {len(all_groups)}")
print(f"  Required fields        : {len(required)}")
print(f"  Fields with aria-desc  : {len(with_instr)}")
print(f"  Orphan labels          : {len(cl02_pay['orphan_labels'])}")
print()
print("  Label source breakdown:")
for src, count in src_counts.most_common():
    flag = "  ⚠" if src in ("placeholder_only", "none") else ""
    print(f"    {src:<22} {count:>4}{flag}")
print()
print(f"  Total payload : {len(cl02_json):>8,} chars  |  ~{cl02_tok:>6,} tokens")

Checklist 02 — Forms summary
  Total forms            : 16
  Total fields           : 16
  Total groups           : 0
  Required fields        : 5
  Fields with aria-desc  : 0
  Orphan labels          : 0

  Label source breakdown:
    label_for                16

  Total payload :    9,975 chars  |  ~ 2,493 tokens


In [19]:
# Deep dive: show concrete examples from each key category
def header(title):
    print(f"\n{'─'*56}")
    print(f"  {title}")
    print(f"{'─'*56}")

header("PLACEHOLDER-ONLY FIELDS  (label disappears on input)")
ph_only = [f for f in all_fields if f["label_source"] == "placeholder_only"]
if ph_only:
    for f in ph_only[:6]:
        print(f"  [{f['type']:<10}] id={str(f['id']):<20}  placeholder={repr(f['placeholder'])}")
else:
    print("  None found.")

header("FIELDS WITH NO ACCESSIBLE NAME")
no_label = [f for f in all_fields if f["label_source"] == "none"]
if no_label:
    for f in no_label[:6]:
        print(f"  [{f['type']:<10}] id={str(f['id']):<20}  name={str(f['name'])}")
else:
    print("  None found.")

header("REQUIRED FIELDS  (first 6)")
for f in required[:6]:
    src = f["label_source"]
    lbl = repr(str(f["effective_label"])[:40])
    print(f"  [{f['type']:<10}] label={lbl:<44}  source={src}")

header("FIELDSET / LEGEND GROUPS")
if all_groups:
    for g in all_groups[:5]:
        leg = repr(g["legend"]) if g["legend"] else "⚠ NO LEGEND"
        print(f"  legend={leg:<40}  inputs={g['input_types']}")
else:
    print("  No fieldset groups found on this page.")

header("FIELDS WITH aria-describedby INSTRUCTIONS")
for f in with_instr[:4]:
    print(f"  [{f['type']:<10}] label={repr(str(f['effective_label'])[:30])}  instructions={repr(str(f['instructions'])[:50])}")


────────────────────────────────────────────────────────
  PLACEHOLDER-ONLY FIELDS  (label disappears on input)
────────────────────────────────────────────────────────
  None found.

────────────────────────────────────────────────────────
  FIELDS WITH NO ACCESSIBLE NAME
────────────────────────────────────────────────────────
  None found.

────────────────────────────────────────────────────────
  REQUIRED FIELDS  (first 6)
────────────────────────────────────────────────────────
  [email     ] label='Subscribe to our mailing list'               source=label_for
  [email     ] label='Subscribe to our mailing list'               source=label_for
  [email     ] label='Subscribe to our mailing list'               source=label_for
  [email     ] label='Subscribe to our mailing list'               source=label_for
  [email     ] label='Subscribe to our mailing list'               source=label_for

────────────────────────────────────────────────────────
  FIELDSET / LEGEND GROUPS
─────

In [20]:
# Token budget per CL02 prompt
placeholder_only_fields = [f for f in all_fields if f["label_source"] == "placeholder_only"]
required_fields         = [f for f in all_fields if f["required"]]
fields_with_instr       = [f for f in all_fields if f["instructions"]]

cl02_slices = {
    "1. Label Quality":         {"forms": [{"fields": [f for f in form["fields"]
                                            if f["label_source"] not in ("placeholder_only", "none")]}
                                           for form in forms]},
    "2. Placeholder-as-Label":  {"fields": placeholder_only_fields},
    "3. Group Label Quality":   {"groups": all_groups},
    "4. Required Indicators":   {"fields": required_fields},
    "5. Instructions Quality":  {"fields": fields_with_instr},
    "6. Overall Form Summary":  cl02_pay,
}

print("Checklist 02 — Token budget per prompt")
print(f"  {'Prompt':<28}  {'Items':>6}  {'~Tokens':>8}")
print(f"  {'-'*48}")
counts = {
    "1. Label Quality":        sum(len(f["fields"]) for f in forms),
    "2. Placeholder-as-Label": len(placeholder_only_fields),
    "3. Group Label Quality":  len(all_groups),
    "4. Required Indicators":  len(required_fields),
    "5. Instructions Quality": len(fields_with_instr),
    "6. Overall Form Summary": len(forms),
}
for name, data in cl02_slices.items():
    t = estimate_tokens(json.dumps(data, indent=2))
    n = counts[name]
    print(f"  {name:<28}  {n:>6}  {t:>8,}")

Checklist 02 — Token budget per prompt
  Prompt                         Items   ~Tokens
  ------------------------------------------------
  1. Label Quality                  16     2,019
  2. Placeholder-as-Label            0         4
  3. Group Label Quality             0         4
  4. Required Indicators             5       550
  5. Instructions Quality            0         4
  6. Overall Form Summary           16     2,493


---
## Checklist 03 — Non-text Content: Images, SVG, Icons & Media

**Extractor**: `nontext_checklist_03.py` | **Prompts**: `nontext_checklist_03.txt`

Images are split into **four categories** because the evaluation criteria differ
for each. Passing the wrong category to the wrong prompt would produce incorrect
assessments.

| Category | Evaluation standard |
|---|---|
| `informative` | Alt text must accurately describe *content* |
| `decorative` | Verify truly decorative — empty alt is correct; informative content with empty alt is a failure |
| `actionable` | Alt/label must describe *destination or action*, not the image appearance |
| `complex` | Needs a long description (aria-describedby or longdesc) for charts/diagrams |

Icon fonts get special treatment: the `sole_content` flag identifies icons that
are the *only* content of a link or button — `aria-hidden="true"` on these makes
the control completely unnamed.

In [21]:
cl03     = load_extractor("nontext_checklist_03")
cl03_pay = cl03.extract(str(HTML_FILE))
cl03_json = json.dumps(cl03_pay, indent=2)
cl03_tok  = estimate_tokens(cl03_json)

img = cl03_pay["images"]
total_images = sum(len(v) for v in img.values())

print("Checklist 03 — Non-text content summary")
print(f"  Images — informative   : {len(img['informative'])}")
print(f"  Images — decorative    : {len(img['decorative'])}")
print(f"  Images — actionable    : {len(img['actionable'])}  (in <a> or <button>)")
print(f"  Images — complex       : {len(img['complex'])}")
print(f"  Total images           : {total_images}")
print(f"  SVGs (non-hidden)      : {len(cl03_pay['svgs'])}")
print(f"  Icon fonts (unique)    : {len(cl03_pay['icon_fonts'])}")
print(f"  Media elements         : {len(cl03_pay['media'])}")
print()

# Pre-flag obvious issues in the data
flagged_imgs  = [i for i in img["informative"] if i["alt_flags"]]
unlabeled_icons = [ic for ic in cl03_pay["icon_fonts"]
                   if ic["sole_content"] and ic["aria_hidden"] and not ic["aria_label"]]
broken_svgs   = [s for s in cl03_pay["svgs"] if not s["title"] and not s["aria_label"]]

print(f"  Pre-detected quality flags:")
print(f"    Informative images with alt_flags  : {len(flagged_imgs)}")
print(f"    Icons: sole_content + aria-hidden  : {len(unlabeled_icons)}  ← unlabeled controls")
print(f"    SVGs with no title or aria-label   : {len(broken_svgs)}")
print()
print(f"  Total payload : {len(cl03_json):>8,} chars  |  ~{cl03_tok:>6,} tokens")

Checklist 03 — Non-text content summary
  Images — informative   : 60
  Images — decorative    : 10
  Images — actionable    : 236  (in <a> or <button>)
  Images — complex       : 0
  Total images           : 306
  SVGs (non-hidden)      : 5
  Icon fonts (unique)    : 12
  Media elements         : 0

  Pre-detected quality flags:
    Informative images with alt_flags  : 0
    Icons: sole_content + aria-hidden  : 1  ← unlabeled controls
    SVGs with no title or aria-label   : 5

  Total payload :   76,663 chars  |  ~19,165 tokens


In [22]:
def header(title):
    print(f"\n{'─'*56}")
    print(f"  {title}")
    print(f"{'─'*56}")

# ── Informative images ────────────────────────────────────────────────────────
header(f"INFORMATIVE IMAGES  ({len(img['informative'])} total — first 6)")
for im in img["informative"][:6]:
    flags = f"  ⚠ {im['alt_flags']}" if im["alt_flags"] else ""
    print(f"  alt={repr(im['alt'][:55])}{flags}")

# ── Decorative images ─────────────────────────────────────────────────────────
header(f"DECORATIVE IMAGES  ({len(img['decorative'])} — LLM verifies they're truly decorative)")
for im in img["decorative"][:5]:
    ctx = repr(im["surrounding_text"][:50]) if im["surrounding_text"] else "None"
    print(f"  src={im['src']:<35}  context={ctx}")

# ── Actionable images ─────────────────────────────────────────────────────────
header(f"ACTIONABLE IMAGES  ({len(img['actionable'])} total — sample 6)")
for im in img["actionable"][:6]:
    lbl = im.get("link_aria_label") or im.get("link_text") or im.get("button_text") or "⚠ NONE"
    print(f"  [{im['context']:<9}] alt={repr(str(im['alt'])[:35]):<38}  parent_label={repr(lbl[:30])}")

# ── SVGs ─────────────────────────────────────────────────────────────────────
header(f"SVGs  ({len(cl03_pay['svgs'])} non-hidden)")
for s in cl03_pay["svgs"]:
    role  = s["role"]  or "⚠ MISSING"
    title = repr(s["title"]) if s["title"] else "⚠ MISSING"
    aria  = repr(s["aria_label"]) if s["aria_label"] else "None"
    print(f"  role={role:<10}  title={title:<30}  aria_label={aria}")

# ── Icon fonts ────────────────────────────────────────────────────────────────
header(f"ICON FONTS  ({len(cl03_pay['icon_fonts'])} unique)")
for ic in cl03_pay["icon_fonts"]:
    problem = ""
    if ic["sole_content"] and ic["aria_hidden"]:
        problem = "  🔴 UNLABELED CONTROL"
    elif not ic["aria_hidden"] and not ic["aria_label"] and not ic["visible_text"]:
        problem = "  ⚠ not hidden, no label"
    hidden_str = "hidden" if ic["aria_hidden"] else "visible"
    sib = repr(ic["sibling_text"][:25]) if ic["sibling_text"] else "None"
    print(f"  [{hidden_str:<7}] sole={str(ic['sole_content']):<5}  "
          f"{ic['classes'][:35]:<35}  sibling={sib}{problem}")


────────────────────────────────────────────────────────
  INFORMATIVE IMAGES  (60 total — first 6)
────────────────────────────────────────────────────────
  alt='Vision-Aid Logo'
  alt='Highlights from the last quarter'
  alt='Mast. Muhamad Farhan'
  alt='With CMC Vellore'
  alt='A girl'
  alt='Mohan Raj’s'

────────────────────────────────────────────────────────
  DECORATIVE IMAGES  (10 — LLM verifies they're truly decorative)
────────────────────────────────────────────────────────
  src=Shakuntala-_poster_fb.jpg            context=None
  src=VisionAidBanner-800x600-1.png        context=None
  src=Shakuntala-_poster_fb.jpg            context=None
  src=VisionAidBanner-800x600-1.png        context=None
  src=Shakuntala-_poster_fb.jpg            context=None

────────────────────────────────────────────────────────
  ACTIONABLE IMAGES  (236 total — sample 6)
────────────────────────────────────────────────────────
  [in_button] alt='Map of India'                          parent_lab

In [23]:
# Token budget per CL03 prompt
cl03_slices = {
    "1. Informative Alt Quality":  {"images": img["informative"]},
    "2. Decorative Verification":  {"images": img["decorative"]},
    "3. Actionable Image Alt":     {"images": img["actionable"]},
    "4. Complex Descriptions":     {"images": img["complex"]},
    "5. SVG Accessibility":        {"svgs":   cl03_pay["svgs"]},
    "6. Icon Font Accessibility":  {"icon_fonts": cl03_pay["icon_fonts"]},
    "7. Media Captions":           {"media":  cl03_pay["media"]},
    "8. Overall Summary":          cl03_pay,
}
item_counts = {
    "1. Informative Alt Quality":  len(img["informative"]),
    "2. Decorative Verification":  len(img["decorative"]),
    "3. Actionable Image Alt":     len(img["actionable"]),
    "4. Complex Descriptions":     len(img["complex"]),
    "5. SVG Accessibility":        len(cl03_pay["svgs"]),
    "6. Icon Font Accessibility":  len(cl03_pay["icon_fonts"]),
    "7. Media Captions":           len(cl03_pay["media"]),
    "8. Overall Summary":          total_images,
}

print("Checklist 03 — Token budget per prompt")
print(f"  {'Prompt':<30}  {'Items':>6}  {'~Tokens':>8}")
print(f"  {'-'*50}")
for name, data in cl03_slices.items():
    t = estimate_tokens(json.dumps(data, indent=2))
    n = item_counts[name]
    print(f"  {name:<30}  {n:>6}  {t:>8,}")

Checklist 03 — Token budget per prompt
  Prompt                           Items   ~Tokens
  --------------------------------------------------
  1. Informative Alt Quality          60     1,562
  2. Decorative Verification          10       325
  3. Actionable Image Alt            236    15,290
  4. Complex Descriptions              0         4
  5. SVG Accessibility                 5       167
  6. Icon Font Accessibility          12       566
  7. Media Captions                    0         4
  8. Overall Summary                 306    19,165


---
## Combined Summary — All Three Checklists

In [24]:
# Side-by-side comparison of all three checklists
checklists = {
    "CL01 Semantic":     (cl01_json, cl01_tok),
    "CL02 Forms":        (cl02_json, cl02_tok),
    "CL03 Non-text":     (cl03_json, cl03_tok),
}
combined_tok = sum(t for _, t in checklists.values())

print("=" * 62)
print("  TOKEN REDUCTION — ALL CHECKLISTS")
print("=" * 62)
print(f"  {'Checklist':<20}  {'Chars':>10}  {'~Tokens':>8}  {'vs raw':>8}")
print(f"  {'-'*54}")

for name, (js, tok) in checklists.items():
    pct_of_raw = tok / raw_tokens * 100
    print(f"  {name:<20}  {len(js):>10,}  {tok:>8,}  {pct_of_raw:>7.1f}%")

print(f"  {'-'*54}")
print(f"  {'COMBINED (all 3)':<20}  {'':>10}  {combined_tok:>8,}  "
      f"{combined_tok/raw_tokens*100:>7.1f}%")
print(f"  {'RAW HTML':<20}  {raw_bytes:>10,}  {raw_tokens:>8,}  {'100.0%':>8}")
print()
reduction = (1 - combined_tok / raw_tokens) * 100
print(f"  Combined token reduction : {reduction:.1f}%")
print(f"  Compression ratio        : {raw_tokens/combined_tok:.0f}× smaller")
print()
print(f"  Even running all 3 checklists together, the combined payload")
print(f"  is {combined_tok:,} tokens — well within a 200k context window.")

  TOKEN REDUCTION — ALL CHECKLISTS
  Checklist                  Chars   ~Tokens    vs raw
  ------------------------------------------------------
  CL01 Semantic             70,644    17,661      3.6%
  CL02 Forms                 9,975     2,493      0.5%
  CL03 Non-text             76,663    19,165      3.9%
  ------------------------------------------------------
  COMBINED (all 3)                    39,319      8.1%
  RAW HTML               1,948,296   486,844    100.0%

  Combined token reduction : 91.9%
  Compression ratio        : 12× smaller

  Even running all 3 checklists together, the combined payload
  is 39,319 tokens — well within a 200k context window.


## Summary

### What Each Checklist Captures

```
Raw HTML (1.9 MB, ~487 k tokens)
        │
        ├──▶ semantic_checklist_01.py  →  Headings, links, landmarks, tables, iframes
        │    semantic_checklist_01.txt    7 prompts — structural + navigation quality
        │
        ├──▶ forms_checklist_02.py     →  Labels, groups, required fields, instructions
        │    forms_checklist_02.txt       6 prompts — label quality + form usability
        │
        └──▶ nontext_checklist_03.py   →  Images (×4 categories), SVG, icons, media
             nontext_checklist_03.txt     8 prompts — alt text + icon/SVG accessibility
```

### Token Summary

| | Raw HTML | CL01 | CL02 | CL03 | All 3 combined |
|---|---|---|---|---|---|
| **Tokens** | ~487,000 | ~17,600 | — | — | — |
| **Reduction** | baseline | ~96% | — | — | — |

*(Run all cells above to fill in CL02 and CL03 figures for this page)*

### Key Findings on This Page (visionaid.org homepage)

From the extractor output above, the LLM prompts will evaluate:

- **CL01**: 280 headings, 76 flagged links, 62 landmarks, 15 iframes — structural
  quality assessment
- **CL02**: 16 forms — label source distribution reveals any placeholder-only
  or unlabeled fields
- **CL03**: 236 actionable images, 5 un-titled SVGs, icon fonts with `sole_content`
  + `aria_hidden` — likely unlabeled controls

### Next Steps

1. **Call the LLM** — iterate over the prompt slices for each checklist, collect JSON
2. **Merge with programmatic results** — combine with `programmatic/semantic_checklist_01.py`
3. **Score and report** — aggregate findings into a WCAG-aligned accessibility report


  home.html
      │
      ├── semantic_checklist_01.py / semantic_checklist_01.txt  ──▶  JSON  ──▶  split into 7 slices  ──▶  7 LLM calls
      ├── forms_checklist_02.py / forms_checklist_02.txt    ──▶  JSON  ──▶  split into 6 slices  ──▶  6 LLM calls
      └── nontext_checklist_03.py   ──▶  JSON  ──▶  split into 8 slices  ──▶  8 LLM calls
                                                                                │
                                                                                ▼
                                                                     merge 21 JSON responses
                                                                                │
                                                                                ▼
                                                                    accessibility report